# Electrochemical Impedance Spectroscopy

This example shows how the Thales EIS software package can be controlled with Python.

The [EIS manual](http://zahner.de/files/eis_2.pdf) provides further explanation of this method and explains the settings.

In [1]:
import sys
from thales_remote.connection import ThalesRemoteConnection
from thales_remote.script_wrapper import PotentiostatMode,ThalesRemoteScriptWrapper

from jupyter_utils import executionInNotebook, notebookCodeToPython

# Establish and initialize 

The Term software must be started before the script is executed to be able to connect.

In [2]:
if __name__ == "__main__":
    zenniumConnection = ThalesRemoteConnection()
    connectionSuccessful = zenniumConnection.connectToTerm("localhost", "ScriptRemote")
    if connectionSuccessful:
        print("connection successfull")
    else:
        print("connection not possible")
        sys.exit()
        
    zahnerZennium = ThalesRemoteScriptWrapper(zenniumConnection)
    zahnerZennium.forceThalesIntoRemoteScript()

connection successfull


# Output file setup

The first step is to set where the measurement data is to be saved. The path must exist otherwise you will get an error.

The measurement results are to be numbered consecutively starting at 13, with the basic filename "spectra".

In [ ]:
    zahnerZennium.setEISOutputPath(r"C:\THALES\temp\test1")
    zahnerZennium.setEISNaming("counter")
    zahnerZennium.setEISCounter(13)
    zahnerZennium.setEISOutputFileName("spectra")

# EIS measurement parameters

In the next step, the actual parameters for the measurement method are set.  
Alternatively, a rule file could be loaded which sets the parameters for the measurement.

In [ ]:
    zahnerZennium.setPotentiostatMode(PotentiostatMode.POTMODE_POTENTIOSTATIC)
    zahnerZennium.setAmplitude(10e-3)
    zahnerZennium.setPotential(1)
    zahnerZennium.setLowerFrequencyLimit(10)
    zahnerZennium.setStartFrequency(1000)
    zahnerZennium.setUpperFrequencyLimit(10000)
    zahnerZennium.setLowerNumberOfPeriods(5)
    zahnerZennium.setLowerStepsPerDecade(2)
    zahnerZennium.setUpperNumberOfPeriods(20)
    zahnerZennium.setUpperStepsPerDecade(5)
    zahnerZennium.setScanDirection("startToMax")
    zahnerZennium.setScanStrategy("single")

# Execute the measurement

The impedance spectrum is now measured at the DC potential of 1.0 V.

<div class="alert alert-block alert-info">
<b>Note:</b> If the potentiostat is set to potentiostatic before the impedance measurement and is switched off, the measurement is performed at the open circuit voltage/potential.
</div>

In [ ]:
    zahnerZennium.enablePotentiostat()
    
    zahnerZennium.measureEIS()
        
    zahnerZennium.disablePotentiostat()

# Changing the potentiostat

By default the main potentiostat with the number 0 is selected. 1 corresponds to the external potentiostat connected to EPC channel 1.

Zahner offers various [External Potentiostats](http://zahner.de/products/external-potentiostats.html) or [Electronic Loads](http://zahner.de/products/electronic-loads.html) with higher power, voltage and current which can be controlled like the internal potentiostat.

In [ ]:
    zahnerZennium.selectPotentiostat(1)

# Configuration of the next output data

For the following measurements, the file name is extended by the date and time.

In [ ]:
    zahnerZennium.setEISNaming("dateTime")
    zahnerZennium.setEISOutputPath(r"C:\THALES\temp\test2")
    zahnerZennium.setEISOutputFileName("spectra")

# EIS measurement parameters

The measurement parameters are reset and the measurement is started.

As the potentiostat was off before the measurement, the measurement is now performed at the open circuit potential/voltage.

In [ ]:
    zahnerZennium.setAmplitude(50e-3)
    zahnerZennium.setScanDirection("startToMin")
    
    zahnerZennium.measureEIS()

# Series measurement with different amplitudes

For the following measurement, the measurement is performed with the different amplitudes defined in the array **AmplitudesIn_mV_forMeasurement**.  
The amplitude is also inserted in the file name.

In [9]:
    zahnerZennium.setEISNaming("individual")
    zahnerZennium.setEISOutputPath(r"C:\THALES\temp\test3")
    
    AmplitudesIn_mV_forMeasurement = [5, 10, 20, 50]
    
    for amplitude in AmplitudesIn_mV_forMeasurement:
        zahnerZennium.setEISOutputFileName("spectraAmplitude{}mV".format(amplitude))
        zahnerZennium.setAmplitude(amplitude / 1000)
        zahnerZennium.measureEIS()

# Disconnect

After the measurements are completed, the device switches back to the main potentiostat and the connection to the term is disconnected.

In [10]:
    zahnerZennium.selectPotentiostat(0)
    
    zenniumConnection.disconnectFromTerm()
    print("finish")

finish


# Deployment of the source code

**The following instruction is not needed by the user.**

It automatically extracts the pure python code from the jupyter notebook to provide it to the user. Thus the user does not need jupyter itself and does not have to copy the code manually.

The source code is saved in a .py file with the same name as the notebook.

In [12]:
    if executionInNotebook() == True:
        notebookCodeToPython("EIS.ipynb")